In [1]:
import re, random, time
import numpy as np
from sklearn.model_selection import train_test_split  # 数据集的划分
import pandas as pd
import pickle
import os, sys
from collections import Counter, defaultdict
import glob
import python_speech_features as ps
import wave
import jieba
import keras as kr

Using TensorFlow backend.


In [2]:
data_all = pickle.load(open("../models/data_word_id_map.pkl","rb"))
id2word, word2id = data_all[4], data_all[5]
wordNum = len(id2word)
df = pd.read_csv('../create_QA_data/qa_data_all.csv')

In [3]:
# 规范化数据，转小写，去除标点等其他非字母符号
def filter_data(s):
    re_han = re.compile("([a-zA-Z]+)")
    s = s.lower()
    blocks = re_han.split(s)
    txt = ''
    for blk in blocks:
        if re_han.match(blk):
            txt = txt + blk + ' '
    return txt

In [4]:
for i in range(len(df)):
    df['Q'][i] = filter_data(df['Q'][i])
    df['A'][i] = filter_data(df['A'][i])

In [5]:
# 8-2比分割训练集和测试集
totalSampleNum = len(df)
testSize = 0.2
trainIdList, testIdList = train_test_split([i for i in range(totalSampleNum)],test_size=testSize)

In [6]:
train_df = df.iloc[trainIdList]
test_df = df.iloc[testIdList]
train_df.index = range(len(train_df))
test_df.index = range(len(test_df))

In [7]:
train_x = np.array(train_df['Q'])
train_x = [i.split() for i in train_x]
train_y = np.array(train_df['A'])
train_y = [i.split() for i in train_y]
train_e = np.array(train_df['AE'])

test_x = np.array(test_df['Q'])
test_x = [i.split() for i in test_x]
test_y = np.array(test_df['A'])
test_y = [i.split() for i in test_y]
test_e = np.array(test_df['AE'])

In [8]:
def get_one_hot(label, wordNum):
    label_arr = [0] * wordNum
    label_arr[label]=1
    return label_arr[:]

In [9]:
def get_word_indices(data_x, MaxLen, types=None):
    length = len(data_x)
    word_list = []
    for word in data_x:
        if not word in word2id:
            # 遇到词汇表之外的词就把它归为unknown
            word_list.append(word2id['<UNK>'])
        else:
            word_list.append(word2id[word])
    if types == 'sos':
        return np.array([word2id['<SOS>']] + word_list + [0]*(MaxLen-length))[:MaxLen]
    elif types == 'eos':
        return np.array(word_list + [word2id['<EOS>']] + [0]*(MaxLen-length))[:MaxLen]
    else:
        return np.array(word_list + [0]*(MaxLen-length))[:MaxLen]

In [10]:
def getEmotionOnehot(e):
    if e == 'ang':
        return np.array([1,0,0,0,0])
    elif e == 'sad':
        return np.array([0,1,0,0,0])
    elif e == 'hap':
        return np.array([0,0,1,0,0])
    elif e == 'neu':
        return np.array([0,0,0,1,0])
    elif e == 'fru':
        return np.array([0,0,0,0,1])
    else:
        return np.array([0,0,0,1,0])

In [11]:
# 构造训练集和测试集的array
MaxLen = 50
train_data = {}
for i in range(len(train_x)):
    question = get_word_indices(train_x[i], MaxLen)
    answer_emotion = getEmotionOnehot(train_e[i])
    # answer1对应decoder_inputs,有<SOS>
    answer1 = get_word_indices(train_y[i], MaxLen, 'sos')
    # answer2对应decoder_outputs,有<EOS>
    answer2 = get_word_indices(train_y[i], MaxLen, 'eos')
    train_data[i]=(question,answer_emotion,answer1,answer2)
    
test_data = {}
for i in range(len(test_x)):
    question = get_word_indices(test_x[i], MaxLen)
    answer_emotion = getEmotionOnehot(test_e[i])
    answer1 = get_word_indices(test_y[i], MaxLen, 'sos')
    answer2 = get_word_indices(test_y[i], MaxLen, 'eos')
    test_data[i]=(question,answer_emotion,answer1,answer2)

In [12]:
def get_emb(local_data, a):
    local_text = []
    for i in range(len(local_data)):
        if a == 'x':
            local_text.append(local_data[i][0])
        elif a == 'y1':
            local_text.append(local_data[i][2])
        elif a == 'y2':
            local_text.append(local_data[i][3])
        elif a == 'e':
            local_text.append(local_data[i][1])
    return np.array(local_text)

In [13]:
train_x = get_emb(train_data, 'x')
train_y1 = get_emb(train_data, 'y1')
train_y2 = get_emb(train_data, 'y2')
train_e = get_emb(train_data, 'e')

test_x = get_emb(test_data, 'x')
test_y1 = get_emb(test_data, 'y1')
test_y2 = get_emb(test_data, 'y2')
test_e = get_emb(test_data, 'e')

In [14]:
def expendToOneHot(x, wordNum, MaxLen):
    t = np.expand_dims(x, axis=2)
    t = t.tolist()
    for i in range(len(t)):
        for j in range(MaxLen):
            print(str(i)+'-'+str(j))
            if(t[i][j][0] == 0):
                t[i][j:] = [[0] * wordNum] * (MaxLen - j)
                break
            else:
                t[i][j] = get_one_hot(t[i][j][0], wordNum)
    return np.array(t)

In [15]:
# 将train_x等扩展1个维度，是word的数量（wordNum维的onehot）
train_x = expendToOneHot(train_x, wordNum, MaxLen)
train_y1 = expendToOneHot(train_y1, wordNum, MaxLen)
train_y2 = expendToOneHot(train_y2, wordNum, MaxLen)

test_x = expendToOneHot(test_x, wordNum, MaxLen)
test_y1 = expendToOneHot(test_y1, wordNum, MaxLen)
test_y2 = expendToOneHot(test_y2, wordNum, MaxLen)

0-0
0-1
0-2
0-3
0-4
0-5
0-6
0-7
0-8
0-9
0-10
0-11
0-12
0-13
0-14
0-15
1-0
1-1
1-2
1-3
1-4
1-5
1-6
1-7
1-8
1-9
1-10
1-11
2-0
2-1
2-2
2-3
2-4
2-5
2-6
3-0
3-1
3-2
3-3
3-4
3-5
3-6
3-7
3-8
3-9
3-10
3-11
3-12
3-13
3-14
3-15
3-16
3-17
3-18
3-19
3-20
3-21
3-22
3-23
3-24
3-25
3-26
4-0
4-1
4-2
4-3
4-4
4-5
4-6
4-7
4-8
4-9
4-10
4-11
4-12
4-13
4-14
4-15
4-16
4-17
4-18
4-19
4-20
4-21
4-22
4-23
4-24
4-25
4-26
4-27
4-28
4-29
4-30
4-31
4-32
4-33
4-34
4-35
4-36
4-37
4-38
4-39
5-0
5-1
5-2
5-3
5-4
5-5
5-6
6-0
6-1
6-2
6-3
6-4
6-5
6-6
6-7
6-8
6-9
6-10
6-11
6-12
6-13
6-14
7-0
7-1
7-2
8-0
8-1
9-0
9-1
9-2
9-3
10-0
10-1
10-2
10-3
10-4
10-5
10-6
10-7
10-8
10-9
10-10
10-11
11-0
11-1
11-2
11-3
11-4
11-5
11-6
11-7
11-8
11-9
11-10
11-11
11-12
11-13
11-14
11-15
11-16
11-17
12-0
12-1
12-2
12-3
12-4
12-5
12-6
12-7
12-8
12-9
12-10
12-11
12-12
12-13
12-14
12-15
12-16
12-17
12-18
12-19
12-20
12-21
12-22
12-23
12-24
12-25
12-26
12-27
12-28
12-29
13-0
13-1
13-2
13-3
14-0
14-1
14-2
14-3
14-4
15-0
15-1
15-2
15-3
15-4
15-5
15-

139-11
139-12
139-13
139-14
139-15
139-16
139-17
139-18
139-19
139-20
140-0
140-1
140-2
140-3
140-4
140-5
141-0
141-1
141-2
141-3
141-4
141-5
141-6
141-7
141-8
141-9
141-10
141-11
141-12
141-13
141-14
141-15
141-16
142-0
142-1
142-2
142-3
142-4
142-5
142-6
142-7
142-8
143-0
143-1
143-2
144-0
144-1
144-2
144-3
144-4
144-5
144-6
144-7
144-8
144-9
144-10
144-11
144-12
144-13
144-14
144-15
144-16
144-17
145-0
145-1
145-2
145-3
146-0
146-1
146-2
146-3
146-4
146-5
146-6
146-7
146-8
146-9
146-10
146-11
146-12
146-13
146-14
146-15
146-16
146-17
146-18
146-19
146-20
146-21
146-22
146-23
146-24
146-25
146-26
146-27
147-0
147-1
147-2
147-3
148-0
148-1
148-2
148-3
148-4
148-5
148-6
148-7
148-8
148-9
148-10
148-11
149-0
149-1
149-2
149-3
149-4
149-5
149-6
149-7
149-8
149-9
149-10
150-0
150-1
150-2
150-3
150-4
150-5
150-6
150-7
150-8
150-9
150-10
150-11
150-12
150-13
150-14
150-15
151-0
151-1
151-2
151-3
151-4
151-5
151-6
151-7
151-8
151-9
151-10
151-11
151-12
151-13
151-14
151-15
151-16
151-17
151-

248-2
248-3
248-4
248-5
248-6
248-7
248-8
248-9
248-10
248-11
248-12
249-0
249-1
249-2
249-3
249-4
249-5
249-6
249-7
249-8
249-9
249-10
249-11
249-12
249-13
249-14
249-15
249-16
249-17
249-18
249-19
249-20
249-21
249-22
249-23
249-24
249-25
249-26
249-27
250-0
250-1
251-0
251-1
251-2
251-3
251-4
251-5
251-6
251-7
251-8
251-9
251-10
251-11
251-12
251-13
251-14
251-15
251-16
251-17
251-18
251-19
251-20
252-0
252-1
252-2
252-3
253-0
253-1
253-2
254-0
254-1
254-2
254-3
254-4
254-5
254-6
254-7
254-8
254-9
254-10
254-11
254-12
254-13
254-14
254-15
254-16
254-17
254-18
254-19
254-20
254-21
254-22
254-23
254-24
254-25
254-26
254-27
254-28
254-29
254-30
254-31
254-32
255-0
255-1
255-2
256-0
256-1
256-2
257-0
257-1
257-2
257-3
257-4
257-5
257-6
257-7
258-0
258-1
259-0
259-1
259-2
259-3
259-4
259-5
259-6
259-7
259-8
259-9
259-10
259-11
259-12
259-13
259-14
259-15
259-16
259-17
259-18
259-19
259-20
259-21
259-22
260-0
260-1
260-2
260-3
260-4
260-5
260-6
260-7
260-8
260-9
260-10
260-11
260-12
260-1

360-5
360-6
360-7
360-8
360-9
360-10
360-11
360-12
360-13
360-14
360-15
360-16
360-17
360-18
360-19
360-20
361-0
361-1
361-2
361-3
361-4
362-0
362-1
362-2
362-3
362-4
362-5
362-6
362-7
362-8
362-9
362-10
362-11
362-12
362-13
362-14
362-15
362-16
362-17
362-18
362-19
362-20
362-21
362-22
362-23
362-24
362-25
362-26
362-27
362-28
362-29
362-30
362-31
363-0
363-1
364-0
364-1
364-2
364-3
364-4
364-5
364-6
364-7
364-8
364-9
364-10
364-11
364-12
364-13
364-14
364-15
364-16
364-17
364-18
364-19
364-20
364-21
364-22
364-23
364-24
364-25
364-26
364-27
364-28
364-29
364-30
364-31
365-0
365-1
365-2
365-3
365-4
365-5
365-6
365-7
365-8
365-9
365-10
365-11
365-12
365-13
365-14
365-15
365-16
365-17
365-18
365-19
365-20
366-0
366-1
366-2
366-3
366-4
366-5
366-6
366-7
366-8
366-9
366-10
366-11
366-12
366-13
366-14
367-0
367-1
367-2
367-3
368-0
368-1
368-2
368-3
368-4
368-5
368-6
368-7
368-8
369-0
369-1
369-2
369-3
369-4
370-0
370-1
370-2
370-3
370-4
370-5
370-6
371-0
371-1
371-2
371-3
371-4
371-5
372-0

465-15
465-16
465-17
466-0
466-1
466-2
466-3
466-4
466-5
466-6
466-7
466-8
466-9
466-10
466-11
466-12
466-13
466-14
466-15
466-16
466-17
466-18
466-19
466-20
466-21
466-22
466-23
466-24
466-25
466-26
466-27
466-28
466-29
466-30
466-31
466-32
466-33
466-34
466-35
467-0
467-1
467-2
467-3
467-4
467-5
467-6
467-7
467-8
467-9
467-10
467-11
467-12
467-13
467-14
467-15
467-16
467-17
467-18
467-19
467-20
467-21
467-22
468-0
468-1
468-2
468-3
468-4
468-5
469-0
469-1
469-2
469-3
469-4
469-5
469-6
469-7
469-8
469-9
470-0
470-1
470-2
470-3
470-4
470-5
470-6
470-7
470-8
470-9
470-10
470-11
470-12
470-13
470-14
470-15
470-16
470-17
470-18
470-19
470-20
470-21
470-22
470-23
470-24
470-25
470-26
470-27
470-28
470-29
470-30
470-31
470-32
470-33
471-0
471-1
471-2
471-3
471-4
471-5
471-6
471-7
471-8
471-9
471-10
471-11
472-0
472-1
472-2
472-3
472-4
472-5
472-6
472-7
472-8
472-9
472-10
472-11
472-12
473-0
473-1
473-2
473-3
473-4
473-5
473-6
473-7
473-8
473-9
473-10
473-11
473-12
473-13
473-14
474-0
474-1


...

127-4
127-5
127-6
127-7
127-8
127-9
127-10
127-11
127-12
127-13
127-14
128-0
128-1
128-2
128-3
128-4
128-5
128-6
128-7
128-8
128-9
128-10
128-11
128-12
128-13
128-14
128-15
128-16
128-17
128-18
129-0
129-1
129-2
129-3
129-4
129-5
129-6
129-7
129-8
129-9
129-10
129-11
129-12
129-13
129-14
129-15
129-16
129-17
129-18
129-19
129-20
129-21
129-22
129-23
129-24
129-25
129-26
130-0
130-1
130-2
130-3
130-4
130-5
130-6
130-7
130-8
130-9
130-10
131-0
131-1
131-2
131-3
131-4
131-5
131-6
131-7
132-0
132-1
132-2
132-3
132-4
132-5
132-6
132-7
132-8
132-9
132-10
132-11
132-12
132-13
132-14
132-15
132-16
132-17
132-18
133-0
133-1
133-2
133-3
133-4
133-5
133-6
133-7
133-8
133-9
133-10
133-11
134-0
134-1
134-2
134-3
134-4
134-5
134-6
134-7
134-8
134-9
134-10
134-11
134-12
134-13
134-14
134-15
134-16
135-0
135-1
135-2
135-3
135-4
135-5
135-6
135-7
135-8
135-9
135-10
135-11
135-12
136-0
136-1
136-2
136-3
136-4
136-5
136-6
136-7
136-8
136-9
136-10
136-11
136-12
136-13
136-14
136-15
136-16
136-17
136-18
13

227-2
227-3
227-4
227-5
227-6
227-7
227-8
228-0
228-1
228-2
228-3
228-4
228-5
228-6
228-7
228-8
228-9
228-10
228-11
228-12
228-13
228-14
228-15
228-16
228-17
228-18
228-19
228-20
229-0
229-1
229-2
229-3
229-4
229-5
229-6
229-7
229-8
230-0
230-1
230-2
230-3
230-4
230-5
230-6
230-7
230-8
230-9
230-10
230-11
230-12
230-13
230-14
231-0
231-1
231-2
231-3
231-4
231-5
231-6
231-7
231-8
231-9
231-10
231-11
231-12
231-13
231-14
231-15
231-16
231-17
231-18
231-19
231-20
231-21
231-22
231-23
231-24
231-25
231-26
231-27
231-28
231-29
231-30
231-31
231-32
231-33
231-34
231-35
231-36
231-37
231-38
231-39
231-40
231-41
231-42
231-43
231-44
231-45
231-46
231-47
231-48
231-49
232-0
232-1
232-2
232-3
232-4
232-5
232-6
232-7
232-8
232-9
232-10
232-11
232-12
232-13
232-14
232-15
233-0
233-1
233-2
233-3
233-4
233-5
233-6
233-7
233-8
233-9
233-10
233-11
233-12
233-13
233-14
233-15
233-16
233-17
233-18
233-19
233-20
233-21
233-22
233-23
233-24
233-25
233-26
233-27
233-28
233-29
233-30
233-31
233-32
233-33
23

326-5
326-6
326-7
326-8
326-9
326-10
326-11
326-12
327-0
327-1
327-2
327-3
327-4
327-5
327-6
328-0
328-1
328-2
328-3
328-4
328-5
328-6
328-7
328-8
328-9
328-10
329-0
329-1
329-2
329-3
329-4
329-5
329-6
329-7
329-8
329-9
329-10
329-11
329-12
329-13
329-14
329-15
329-16
329-17
329-18
329-19
329-20
329-21
329-22
329-23
329-24
329-25
329-26
329-27
329-28
329-29
329-30
329-31
329-32
330-0
330-1
330-2
330-3
330-4
330-5
330-6
330-7
330-8
330-9
330-10
330-11
331-0
331-1
331-2
331-3
331-4
331-5
331-6
331-7
331-8
331-9
331-10
331-11
331-12
332-0
332-1
332-2
332-3
332-4
332-5
332-6
332-7
332-8
332-9
332-10
332-11
332-12
332-13
332-14
333-0
333-1
333-2
333-3
333-4
333-5
333-6
333-7
333-8
334-0
334-1
334-2
334-3
334-4
334-5
334-6
334-7
334-8
334-9
334-10
334-11
334-12
335-0
335-1
335-2
335-3
335-4
335-5
335-6
335-7
335-8
335-9
335-10
335-11
335-12
335-13
335-14
335-15
335-16
335-17
335-18
336-0
336-1
336-2
336-3
336-4
336-5
336-6
336-7
336-8
336-9
336-10
336-11
337-0
337-1
337-2
337-3
337-4
337-5
3

415-0
415-1
415-2
415-3
416-0
416-1
416-2
416-3
416-4
417-0
417-1
417-2
417-3
417-4
417-5
417-6
418-0
418-1
418-2
418-3
418-4
418-5
418-6
418-7
419-0
419-1
419-2
419-3
419-4
419-5
419-6
419-7
419-8
419-9
419-10
419-11
419-12
419-13
419-14
419-15
420-0
420-1
420-2
420-3
420-4
420-5
420-6
420-7
420-8
420-9
420-10
420-11
420-12
421-0
421-1
421-2
421-3
421-4
421-5
421-6
421-7
421-8
421-9
422-0
422-1
422-2
422-3
422-4
423-0
423-1
423-2
423-3
423-4
423-5
423-6
423-7
423-8
423-9
423-10
424-0
424-1
424-2
425-0
425-1
425-2
425-3
425-4
425-5
425-6
425-7
425-8
425-9
425-10
425-11
425-12
425-13
425-14
425-15
425-16
425-17
425-18
425-19
425-20
426-0
426-1
426-2
426-3
426-4
426-5
427-0
427-1
427-2
427-3
427-4
427-5
427-6
427-7
427-8
427-9
427-10
427-11
427-12
427-13
428-0
428-1
428-2
428-3
428-4
429-0
429-1
429-2
429-3
429-4
430-0
430-1
430-2
430-3
430-4
430-5
431-0
431-1
431-2
432-0
432-1
432-2
432-3
433-0
433-1
433-2
433-3
433-4
433-5
433-6
433-7
433-8
433-9
433-10
433-11
433-12
433-13
433-14
433-

519-20
519-21
519-22
519-23
520-0
520-1
520-2
520-3
520-4
520-5
520-6
520-7
520-8
520-9
520-10
520-11
521-0
521-1
521-2
521-3
521-4
522-0
522-1
522-2
522-3
522-4
522-5
522-6
522-7
522-8
522-9
522-10
522-11
522-12
522-13
522-14
522-15
522-16
522-17
522-18
522-19
522-20
522-21
522-22
522-23
522-24
522-25
522-26
522-27
522-28
522-29
522-30
522-31
522-32
522-33
522-34
522-35
522-36
522-37
522-38
522-39
522-40
522-41
522-42
522-43
522-44
522-45
522-46
522-47
522-48
522-49
523-0
523-1
523-2
523-3
523-4
523-5
523-6
523-7
523-8
523-9
523-10
523-11
523-12
523-13
523-14
523-15
523-16
523-17
523-18
523-19
523-20
523-21
523-22
523-23
523-24
523-25
523-26
523-27
523-28
523-29
523-30
523-31
524-0
524-1
524-2
524-3
524-4
524-5
524-6
524-7
524-8
524-9
524-10
524-11
524-12
524-13
524-14
524-15
524-16
524-17
524-18
524-19
524-20
524-21
524-22
524-23
524-24
524-25
524-26
524-27
524-28
524-29
524-30
524-31
524-32
524-33
524-34
524-35
524-36
524-37
524-38
524-39
524-40
524-41
524-42
525-0
525-1
525-2
525-3

In [16]:
f = open('../models/train_test_emotion_data.pkl', 'wb')
pickle.dump((train_x, train_y1, train_y2, train_e, test_x, test_y1, test_y2, test_e), f)
f.close()
pass